In [17]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('data_final.csv')

# Binary target: up/down movement
data['Direction'] = (data['LogReturn'] > 0).astype(int)

y = data['Direction']
X = data.drop(['LogReturn', 'Direction', 'date'], axis=1, errors='ignore')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Attention layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )
    return model

# Hyperparameter Tuning (TimeSeriesSplit CV)
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=15,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='classification'
)

tscv = TimeSeriesSplit(n_splits=5)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=100,
        batch_size=64,
        callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=10), keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=5)],
        verbose=0
    )

# Retrieve and train best model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

# Fix: Don't use the get() method with default values
# Instead, use fixed values for epochs and batch_size
best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,  # Fixed value instead of best_hp.get('tuner/epochs', 50)
    batch_size=64,  # Fixed value instead of best_hp.get('tuner/batch_size', 64)
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Evaluate
loss, acc, auc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Accuracy: {acc:.4f}, AUC: {auc:.4f}")

# Detailed metrics
y_pred = (best_model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Save model
best_model.save('best_classification_model.keras')
print("Saved classification model")

Reloading Tuner from tuner_dir/classification/tuner0.json
Epoch 1/50


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.5819 - auc: 0.5856 - loss: 0.6792 - val_accuracy: 0.5029 - val_auc: 0.5009 - val_loss: 0.7010
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6204 - auc: 0.6444 - loss: 0.6582 - val_accuracy: 0.5000 - val_auc: 0.5075 - val_loss: 0.7089
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6246 - auc: 0.6715 - loss: 0.6517 - val_accuracy: 0.5115 - val_auc: 0.5122 - val_loss: 0.7160
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6119 - auc: 0.6614 - loss: 0.6514 - val_accuracy: 0.5374 - val_auc: 0.5215 - val_loss: 0.7221
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6427 - auc: 0.7059 - loss: 0.6352 - val_accuracy: 0.5086 - val_auc: 0.5290 - val_loss: 0.7431
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6582 - auc: 0.7440 - loss: 0.6082 - val_accuracy: 0.5115 - val_auc: 0.5459 - val_loss: 0.7569
Test Loss: 0.8666, Accuracy: 0.4516, AUC: 0.4580
14/14 ━━

In [21]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('data_final.csv')

# Binary target: up/down movement
data['Direction'] = (data['LogReturn'] > 0).astype(int)

y = data['Direction']
X = data.drop(['LogReturn', 'Direction', 'date'], axis=1, errors='ignore')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Attention layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )
    return model

# Hyperparameter Tuning (TimeSeriesSplit CV)
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=15,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='classification'
)

tscv = TimeSeriesSplit(n_splits=5)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=100,
        batch_size=64,
        callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=10), keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=5)],
        verbose=0
    )

# Retrieve and train best model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

# Fix: Don't use the get() method with default values
# Instead, use fixed values for epochs and batch_size
best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,  # Fixed value instead of best_hp.get('tuner/epochs', 50)
    batch_size=64,  # Fixed value instead of best_hp.get('tuner/batch_size', 64)
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Print model summary
print("\n=== Model Architecture ===")
best_model.summary()

# Evaluate
loss, acc, auc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {loss:.4f}, Accuracy: {acc:.4f}, AUC: {auc:.4f}")

# Detailed metrics
y_pred = (best_model.predict(X_test) > 0.5).astype(int)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

# Feature Importance using custom approach
print("\n=== Feature Importance Analysis ===")

# Get feature names
feature_names = X.columns if hasattr(X, 'columns') else [f"Feature_{i}" for i in range(X.shape[1])]

# Analyze feature importance at the most recent time point (most relevant for prediction)
print("Analyzing feature importance at most recent time point...")
X_recent = X_test[:, -1, :]  # Most recent data point in each sequence

# Manual implementation of permutation importance
n_features = X_recent.shape[1]
n_repeats = 5
importances = np.zeros(n_features)
importances_std = np.zeros(n_features)

# Get baseline score with original data
baseline_preds = best_model.predict(X_test, verbose=0)
baseline_score = roc_auc_score(y_test, baseline_preds)
print(f"Baseline ROC AUC: {baseline_score:.4f}")

# Compute feature importance
for feature_idx in range(n_features):
    print(f"Computing importance for feature {feature_idx+1}/{n_features}: {feature_names[feature_idx] if feature_idx < len(feature_names) else 'Feature_'+str(feature_idx)}")
    
    feature_scores = []
    
    # Repeat permutation multiple times
    for _ in range(n_repeats):
        # Create a copy of the test data
        X_test_permuted = X_test.copy()
        
        # Permute the values of the feature at the most recent time step
        permuted_values = X_recent[:, feature_idx].copy()
        np.random.shuffle(permuted_values)
        X_test_permuted[:, -1, feature_idx] = permuted_values
        
        # Get predictions with permuted feature
        permuted_preds = best_model.predict(X_test_permuted, verbose=0)
        
        # Calculate score with permuted feature
        permuted_score = roc_auc_score(y_test, permuted_preds)
        
        # Calculate importance as decrease in performance
        importance = baseline_score - permuted_score
        feature_scores.append(importance)
    
    # Store mean and std of importance scores
    importances[feature_idx] = np.mean(feature_scores)
    importances_std[feature_idx] = np.std(feature_scores)



# Sort features by importance
indices = np.argsort(importances)[::-1]

# Print feature ranking
print("\nFeature ranking by importance:")
for i, idx in enumerate(indices):
    if i < 20:  # Print top 20 features
        feature_name = feature_names[idx] if idx < len(feature_names) else f"Feature_{idx}"
        print(f"{i+1}. {feature_name}: {importances[idx]:.4f} Â± {importances_std[idx]:.4f}")

# Visualize feature importance
plt.figure(figsize=(12, 8))
plt.title("Feature Importance (Top 15)")
top_indices = indices[:15]
plt.barh(range(len(top_indices)), importances[top_indices], color="r", yerr=importances_std[top_indices], align="center")
plt.yticks(range(len(top_indices)), [feature_names[i] if i < len(feature_names) else f"Feature_{i}" for i in top_indices])
plt.ylim([-1, len(top_indices)])
plt.xlabel("Feature Importance (Mean Decrease in AUC)")
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()
print("Saved feature importance visualization to 'feature_importance.png'")

# Save model
best_model.save('best_classification_model.keras')
print("\nSaved classification model")

Reloading Tuner from tuner_dir/classification/tuner0.json
Epoch 1/50


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.5397 - auc: 0.5571 - loss: 0.6870 - val_accuracy: 0.4770 - val_auc: 0.5177 - val_loss: 0.7002
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6126 - auc: 0.6255 - loss: 0.6627 - val_accuracy: 0.4799 - val_auc: 0.5226 - val_loss: 0.7062
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6182 - auc: 0.6564 - loss: 0.6529 - val_accuracy: 0.5000 - val_auc: 0.5343 - val_loss: 0.7129
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6555 - auc: 0.7032 - loss: 0.6293 - val_accuracy: 0.4885 - val_auc: 0.5468 - val_loss: 0.7434
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.6565 - auc: 0.7184 - loss: 0.6231 - val_accuracy: 0.5086 - val_auc: 0.5585 - val_loss: 0.7556
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6645 - auc: 0.7210 - loss: 0.6245 - val_accuracy: 0.4914 - val_auc: 0.5588 - val_loss: 0.7952

=== Model Architecture ===


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_40                │ (None, 30, 128)        │        45,568 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_41                │ (None, 30, 128)        │        74,496 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_20 (Attention)        │ (None, 128)            │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 460,421 (1.76 MB)

 Trainable params: 153,473 (599.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 306,948 (1.17 MB)


Test Loss: 0.9598, Accuracy: 0.4447, AUC: 0.4646
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.45      0.86      0.59       200
           1       0.43      0.09      0.15       234

    accuracy                           0.44       434
   macro avg       0.44      0.47      0.37       434
weighted avg       0.44      0.44      0.35       434


=== Confusion Matrix ===
[[172  28]
 [213  21]]

=== Feature Importance Analysis ===
Analyzing feature importance at most recent time point...
Baseline ROC AUC: 0.4649
Computing importance for feature 1/24: nonfarm_payrolls
Computing importance for feature 2/24: corporate_profits
Computing importance for feature 3/24: consumer_confidence
Computing importance for feature 4/24: permits
Computing importance for feature 5/24: unemployment_lag1
Computing importance for feature 6/24: interest_rate_roll3_std
Computing importance for feature 7/24: Gold_